In [1]:
from load_data_json import LoadDataset 
from preprocess import VectorSpaceModel
from preprocess_restart import VectorSpaceModel2
import numpy as np
import time
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds, eigsh
import ipyparallel as ipp
from scipy.sparse import diags
import math


## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./data/med/med.json", "./data/med/queries.json", "./data/med/qrels-treceval.txt","med")
# vsm = VectorSpaceModel(dataset.doc_matrix, dataset.name)
vsm = VectorSpaceModel2(dataset.doc_matrix, dataset.name)

c:\Users\dungl\anaconda3\envs\IR_VectorSpaceModel\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [38]:
def bisec_PDDP(A, iter = 4):
        # if np.all(indices==None):
        #     A = self.A
        # elif self.left:
        #     A = self.A[indices, :]
        # else:
        #     A = self.A[:, indices]
        
        m, n = A.shape

        # if m < n:
        #     d = np.sum(A, axis=0)/m
        #     q = np.zeros((m, iter))
        #     v = np.random.rand(m)
        # else:

        d = np.sum(A, axis=1)/n
        q = np.zeros((n, iter+1))
        v = np.random.rand(n)
        
        q[:,0] = v/np.linalg.norm(v)
        d = np.array(d).flatten()
        
        # lanczos
        alpha = np.zeros(iter+1)
        beta = np.zeros(iter+1)
        
        for i in range(iter):
            # if m < n:
            #     q_hat = A.T @ q[:,i] - np.sum(q[:,i]) * d
            #     w = A @ q_hat - beta[i] * q[:,i-1] - np.full(m, q_hat @ d)
            # else:
            q_hat = A @ q[:,i] - np.sum(q[:,i]) * d
            w = A.T @ q_hat - beta[i] * q[:,i-1] - np.full(n, q_hat @ d)

            alpha[i] = w.dot(q[:,i])
            w = w - alpha[i] * q[:,i]

            beta[i+1] = np.linalg.norm(w)
            if beta[i+1] == 0:
                break
            q[:,i+1] = w / beta[i+1]
        # end lanczos
        T = diags([alpha, beta, beta], [0, -1, 1]).toarray()
        w, v = eigsh(T)
        print(v.shape)
        principal = q.dot(v[:,0])

       
        lo_bound = np.quantile(principal,.45)
        up_bound = np.quantile(principal,.55)
        left = np.where(principal < up_bound)[0]
        right = np.where(principal > lo_bound)[0]
        return left, right
        # if np.all(indices==None):
        #     return left, right
        # else:
        #     return indices[left], indices[right]

In [39]:
l, r = bisec_PDDP(dataset.doc_matrix)

(5, 5)


C:\Users\dungl\AppData\Local\Temp\ipykernel_13236\375018530.py:45: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  w, v = eigsh(T)


In [37]:
r

array([   0,    2,    3,    4,    5,    6,    7,    9,   12,   14,   15,
         18,   19,   21,   22,   25,   26,   27,   29,   30,   33,   34,
         35,   38,   39,   41,   42,   44,   46,   47,   48,   49,   50,
         51,   52,   53,   55,   56,   57,   61,   62,   63,   67,   71,
         75,   79,   80,   81,   82,   83,   84,   87,   88,   89,   90,
         95,   96,   97,   99,  100,  101,  102,  103,  104,  105,  106,
        108,  109,  110,  111,  112,  113,  114,  115,  116,  117,  119,
        121,  123,  124,  125,  133,  134,  135,  136,  137,  138,  139,
        140,  141,  142,  145,  147,  148,  149,  151,  156,  157,  158,
        160,  161,  163,  164,  166,  167,  168,  169,  170,  179,  180,
        182,  183,  185,  186,  192,  193,  197,  198,  199,  200,  201,
        202,  203,  205,  210,  211,  212,  214,  216,  224,  226,  227,
        228,  230,  234,  235,  236,  237,  238,  239,  240,  241,  242,
        243,  244,  252,  255,  257,  259,  262,  2

In [32]:
r

array([   0,    4,    5,    6,    7,    9,   11,   15,   16,   17,   22,
         24,   26,   27,   28,   29,   30,   31,   32,   33,   35,   38,
         42,   44,   45,   48,   49,   50,   52,   55,   56,   62,   63,
         65,   67,   73,   74,   75,   76,   81,   83,   85,   86,   87,
         88,   91,   96,   99,  100,  101,  102,  103,  104,  105,  106,
        109,  110,  111,  112,  113,  114,  115,  116,  117,  118,  119,
        120,  121,  123,  124,  125,  131,  133,  135,  138,  139,  140,
        145,  147,  148,  149,  150,  153,  156,  157,  158,  160,  173,
        186,  187,  191,  199,  202,  203,  204,  215,  216,  217,  218,
        219,  221,  223,  224,  225,  227,  228,  230,  237,  238,  239,
        240,  241,  242,  244,  245,  246,  247,  248,  251,  252,  259,
        260,  266,  267,  271,  272,  275,  278,  287,  292,  297,  302,
        304,  305,  306,  307,  308,  309,  310,  311,  312,  313,  315,
        316,  317,  318,  319,  320,  321,  322,  3

In [19]:

A

matrix([[-0.0003274 , -0.0003274 , -0.0003274 , ..., -0.0003274 ,
         -0.0003274 , -0.0003274 ],
        [-0.00793646, -0.00793646,  0.04057048, ...,  0.02286208,
         -0.00793646, -0.00793646],
        [-0.00672568, -0.00672568,  0.04696494, ..., -0.00672568,
         -0.00672568, -0.00672568],
        ...,
        [-0.00011589, -0.00011589, -0.00011589, ..., -0.00011589,
         -0.00011589, -0.00011589],
        [-0.00010179, -0.00010179, -0.00010179, ..., -0.00010179,
         -0.00010179, -0.00010179],
        [-0.00029107, -0.00029107, -0.00029107, ..., -0.00029107,
         -0.00029107, -0.00029107]])

In [3]:
def bisec_PDDP(A):
    n = A.shape[1]
    d = np.sum(A, axis=1)
    A = A - d/n
    print
    u, s, vt = TruncatedSVD(A)
    principal = vt.T[:,0]

In [5]:
bisec_PDDP(dataset.doc_matrix)

TypeError: cannot unpack non-iterable TruncatedSVD object